In [1]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

PyTorch version: 2.5.1+cu124
Torchvision version: 0.20.1+cu124
CUDA is available: False


/home/zhongyi/anaconda3/envs/sam/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )
np.random.seed(3)


using device: cpu


First, load the SAM 2 model and predictor. Change the path below to point to the SAM 2 checkpoint. Running on CUDA and using the default model are recommended for best results.

In [3]:
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

sam2_checkpoint = "../checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=device)

predictor = SAM2ImagePredictor(sam2_model)

In [4]:
import numpy as np
from scipy.ndimage import label, find_objects, binary_erosion, binary_dilation
from scipy.ndimage import generate_binary_structure


def get_largest_connected_component_and_bbox_mask(mask, radius=5, dilation_radius=20):
    # Define a structuring element for erosion and dilation
    struct_elem = generate_binary_structure(2, 1)
    
    # Erode and then dilate the mask to remove small noise
    mask_eroded = binary_erosion(mask, structure=struct_elem, iterations=radius)
    mask_cleaned = binary_dilation(mask_eroded, structure=struct_elem, iterations=radius)
    
    # Label connected components in the cleaned mask
    labeled_array, num_features = label(mask_cleaned)
    
    # Get sizes of connected components
    component_sizes = [(labeled_array == i).sum() for i in range(1, num_features + 1)]
    
    # Identify the largest component by its label
    largest_component_label = np.argmax(component_sizes) + 1
    
    # Create a mask with only the largest connected component
    largest_component = (labeled_array == largest_component_label).astype(np.uint8)
    
    # Find the bounding box of the largest connected component
    bbox = find_objects(labeled_array == largest_component_label)[0]
    min_row, min_col = bbox[0].start, bbox[1].start
    max_row, max_col = bbox[0].stop, bbox[1].stop
    
    # Create an empty array of the same shape as the input mask
    bbox_mask = np.zeros_like(mask, dtype=np.uint8)
    
    # Set the bounding box region to 1
    bbox_mask[min_row:max_row, min_col:max_col] = 1

    # Dilate the bounding box mask by 20 pixels
    bbox_mask_dilated = binary_dilation(bbox_mask, structure=struct_elem, iterations=dilation_radius)
    
    return largest_component, bbox_mask_dilated


def crop_image_by_bbox_per_channel(image, bbox_mask):
    # Find the coordinates of the bounding box from the bbox mask
    rows = np.any(bbox_mask, axis=1)
    cols = np.any(bbox_mask, axis=0)
    min_row, max_row = np.where(rows)[0][[0, -1]]
    min_col, max_col = np.where(cols)[0][[0, -1]]
    
    # Crop each channel of the image using the bounding box coordinates
    cropped_channels = [image[min_row:max_row+1, min_col:max_col+1, c] for c in range(image.shape[2])]
    
    # Stack cropped channels back together to form the final cropped image
    cropped_image = np.stack(cropped_channels, axis=-1)
    
    return cropped_image

def get_center_and_center_rectangle(array, size_factor=0.2):
    h, w = array.shape
    cy, cx = h // 2, w // 2
    rh, rw = int(h * size_factor), int(w * size_factor)
    
    # Calculate the corner points
    top_left = [cx - rw // 2, cy - rh // 2]
    top_right = [top_left[0], top_left[1] + rh]
    bottom_left = [top_left[0] + rw, top_left[1]]
    bottom_right = [top_left[0] + rw, top_left[1] + rh]
    
    # Return the coordinates as a list
    return np.array([[cx, cy], top_left, top_right, bottom_left, bottom_right])



## Example image

In [5]:
from PIL import Image

# Directories for input images and output cropped images
img_dir  = 'quarter_reslution_tactile_img_717/'  # Directory containing the original tactile images
crop_dir = 'quarter_reslution_tactile_img_cropped/'  # Directory to save cropped images

# Iterate through all image files in the input directory, sorted alphabetically
for idx, d in enumerate(sorted(os.listdir(img_dir))):
    # Construct the full path to the input image
    img_path = img_dir + d    
    
    # Modify the file name to append '_cropped' before the file extension
    dot_index = d.rfind('.')  # Find the position of the file extension
    crop_d = d[:dot_index] + '_cropped' + d[dot_index:]  # Add '_cropped' to the file name
    targ_path = crop_dir + crop_d  # Full path for the cropped image output
    
    # Log the index and current file name being processed
    print(idx, d)
    
    # Open the image and convert it to an RGB numpy array
    image = Image.open(img_path)
    image = np.array(image.convert("RGB"))  # Ensure the image is in RGB format
    
    # Set the current image for the SAM2 predictor
    predictor.set_image(image)
    
    # Obtain the central point and central rectangle of the image for generating mask prompts
    input_point = get_center_and_center_rectangle(image[:, :, 0])  # Central coordinates as points
    input_label = np.array([1, 1, 1, 1, 1])  # Labels for each point (indicating 1-class foreground )
    
    # Use the predictor to generate segmentation masks
    masks, scores, logits = predictor.predict(
        point_coords=input_point,  # Input points for segmentation
        point_labels=input_label,  # Labels for the input points
        multimask_output=True,  # Generate multiple mask outputs
    )
    
    # Sort the masks by their confidence scores in descending order
    sorted_ind = np.argsort(scores)[::-1]  # Get indices of scores in descending order
    masks = masks[sorted_ind]  # Reorder masks by score
    scores = scores[sorted_ind]  # Reorder scores
    logits = logits[sorted_ind]  # Reorder logits
    
    # Extract the largest connected component and its bounding box mask from the best mask
    largest_component, bbox_mask = get_largest_connected_component_and_bbox_mask(masks[0])
    
    # Crop the image based on the bounding box mask
    cropped_image = crop_image_by_bbox_per_channel(image, bbox_mask)
    
    # Ensure the cropped image is in the correct format for saving (uint8)
    cropped_image = (cropped_image * 255).astype(np.uint8) if cropped_image.dtype != np.uint8 else cropped_image
    
    # Save the cropped image to the target path
    Image.fromarray(cropped_image).save(targ_path)
    
    # Log the success of the cropping operation, including the shape of the cropped image
    print(idx, d, "Cropped image shape:", cropped_image.shape, 'saved', targ_path)


0 ID_001_P1099730.JPG
0 ID_001_P1099730.JPG Cropped image shape: (2092, 1063, 3) saved quarter_reslution_tactile_img_cropped/ID_001_P1099730_cropped.JPG
1 ID_001_P1099731.JPG
1 ID_001_P1099731.JPG Cropped image shape: (2341, 1788, 3) saved quarter_reslution_tactile_img_cropped/ID_001_P1099731_cropped.JPG
2 ID_001_P1099732.JPG
2 ID_001_P1099732.JPG Cropped image shape: (2339, 1785, 3) saved quarter_reslution_tactile_img_cropped/ID_001_P1099732_cropped.JPG
3 ID_001_P1099734.JPG
3 ID_001_P1099734.JPG Cropped image shape: (2025, 1058, 3) saved quarter_reslution_tactile_img_cropped/ID_001_P1099734_cropped.JPG
4 ID_001_P1099735.JPG
4 ID_001_P1099735.JPG Cropped image shape: (2338, 1788, 3) saved quarter_reslution_tactile_img_cropped/ID_001_P1099735_cropped.JPG
5 ID_001_P1099736.JPG
5 ID_001_P1099736.JPG Cropped image shape: (2043, 1162, 3) saved quarter_reslution_tactile_img_cropped/ID_001_P1099736_cropped.JPG
6 ID_001_P1099737.JPG
6 ID_001_P1099737.JPG Cropped image shape: (1911, 937, 3) sa

53 ID_010_IMG_6354.jpeg Cropped image shape: (2603, 3703, 3) saved quarter_reslution_tactile_img_cropped/ID_010_IMG_6354_cropped.jpeg
54 ID_010_IMG_6355.jpeg
54 ID_010_IMG_6355.jpeg Cropped image shape: (2634, 3656, 3) saved quarter_reslution_tactile_img_cropped/ID_010_IMG_6355_cropped.jpeg
55 ID_012_P1099772.JPG
55 ID_012_P1099772.JPG Cropped image shape: (2233, 1665, 3) saved quarter_reslution_tactile_img_cropped/ID_012_P1099772_cropped.JPG
56 ID_012_P1099773.JPG
56 ID_012_P1099773.JPG Cropped image shape: (2247, 1666, 3) saved quarter_reslution_tactile_img_cropped/ID_012_P1099773_cropped.JPG
57 ID_012_P1099774.JPG
57 ID_012_P1099774.JPG Cropped image shape: (2228, 1666, 3) saved quarter_reslution_tactile_img_cropped/ID_012_P1099774_cropped.JPG
58 ID_012_P1099775.JPG
58 ID_012_P1099775.JPG Cropped image shape: (2227, 1665, 3) saved quarter_reslution_tactile_img_cropped/ID_012_P1099775_cropped.JPG
59 ID_012_P1099776.JPG
59 ID_012_P1099776.JPG Cropped image shape: (2229, 1662, 3) saved

106 ID_017_P1099856.JPG Cropped image shape: (2039, 1429, 3) saved quarter_reslution_tactile_img_cropped/ID_017_P1099856_cropped.JPG
107 ID_017_P1099857.JPG
107 ID_017_P1099857.JPG Cropped image shape: (2041, 1449, 3) saved quarter_reslution_tactile_img_cropped/ID_017_P1099857_cropped.JPG
108 ID_017_P1099858.JPG
108 ID_017_P1099858.JPG Cropped image shape: (2055, 1480, 3) saved quarter_reslution_tactile_img_cropped/ID_017_P1099858_cropped.JPG
109 ID_018_P1099821.JPG
109 ID_018_P1099821.JPG Cropped image shape: (2221, 1537, 3) saved quarter_reslution_tactile_img_cropped/ID_018_P1099821_cropped.JPG
110 ID_018_P1099822.JPG
110 ID_018_P1099822.JPG Cropped image shape: (2224, 1540, 3) saved quarter_reslution_tactile_img_cropped/ID_018_P1099822_cropped.JPG
111 ID_018_P1099823.JPG
111 ID_018_P1099823.JPG Cropped image shape: (2223, 1551, 3) saved quarter_reslution_tactile_img_cropped/ID_018_P1099823_cropped.JPG
112 ID_018_P1099824.JPG
112 ID_018_P1099824.JPG Cropped image shape: (2220, 1607, 

159 ID_023_P1099972.JPG Cropped image shape: (2025, 1445, 3) saved quarter_reslution_tactile_img_cropped/ID_023_P1099972_cropped.JPG
160 ID_023_P1099973.JPG
160 ID_023_P1099973.JPG Cropped image shape: (2040, 1456, 3) saved quarter_reslution_tactile_img_cropped/ID_023_P1099973_cropped.JPG
161 ID_023_P1099974.JPG
161 ID_023_P1099974.JPG Cropped image shape: (2022, 1420, 3) saved quarter_reslution_tactile_img_cropped/ID_023_P1099974_cropped.JPG
162 ID_023_P1099975.JPG
162 ID_023_P1099975.JPG Cropped image shape: (2039, 1452, 3) saved quarter_reslution_tactile_img_cropped/ID_023_P1099975_cropped.JPG
163 ID_024_P1099872.JPG
163 ID_024_P1099872.JPG Cropped image shape: (2064, 1465, 3) saved quarter_reslution_tactile_img_cropped/ID_024_P1099872_cropped.JPG
164 ID_024_P1099873.JPG
164 ID_024_P1099873.JPG Cropped image shape: (2064, 1464, 3) saved quarter_reslution_tactile_img_cropped/ID_024_P1099873_cropped.JPG
165 ID_024_P1099874.JPG
165 ID_024_P1099874.JPG Cropped image shape: (2064, 1465, 

212 ID_029_P1099915.JPG Cropped image shape: (2038, 1449, 3) saved quarter_reslution_tactile_img_cropped/ID_029_P1099915_cropped.JPG
213 ID_029_P1099916.JPG
213 ID_029_P1099916.JPG Cropped image shape: (2038, 1448, 3) saved quarter_reslution_tactile_img_cropped/ID_029_P1099916_cropped.JPG
214 ID_029_P1099917.JPG
214 ID_029_P1099917.JPG Cropped image shape: (2038, 1441, 3) saved quarter_reslution_tactile_img_cropped/ID_029_P1099917_cropped.JPG
215 ID_029_P1099918.JPG
215 ID_029_P1099918.JPG Cropped image shape: (2038, 1437, 3) saved quarter_reslution_tactile_img_cropped/ID_029_P1099918_cropped.JPG
216 ID_029_P1099919.JPG
216 ID_029_P1099919.JPG Cropped image shape: (2057, 1463, 3) saved quarter_reslution_tactile_img_cropped/ID_029_P1099919_cropped.JPG
217 ID_030_P1099920.JPG
217 ID_030_P1099920.JPG Cropped image shape: (2058, 1463, 3) saved quarter_reslution_tactile_img_cropped/ID_030_P1099920_cropped.JPG
218 ID_030_P1099921.JPG
218 ID_030_P1099921.JPG Cropped image shape: (2063, 1460, 

...

476 ID_072_P1100151.JPG Cropped image shape: (1884, 1319, 3) saved quarter_reslution_tactile_img_cropped/ID_072_P1100151_cropped.JPG
477 ID_073_P1100152.JPG
477 ID_073_P1100152.JPG Cropped image shape: (1856, 1319, 3) saved quarter_reslution_tactile_img_cropped/ID_073_P1100152_cropped.JPG
478 ID_073_P1100153.JPG
478 ID_073_P1100153.JPG Cropped image shape: (1855, 1319, 3) saved quarter_reslution_tactile_img_cropped/ID_073_P1100153_cropped.JPG
479 ID_073_P1100154.JPG
479 ID_073_P1100154.JPG Cropped image shape: (1856, 1320, 3) saved quarter_reslution_tactile_img_cropped/ID_073_P1100154_cropped.JPG
480 ID_073_P1100155.JPG
480 ID_073_P1100155.JPG Cropped image shape: (1864, 1322, 3) saved quarter_reslution_tactile_img_cropped/ID_073_P1100155_cropped.JPG
481 ID_073_P1100156.JPG
481 ID_073_P1100156.JPG Cropped image shape: (1865, 1315, 3) saved quarter_reslution_tactile_img_cropped/ID_073_P1100156_cropped.JPG
482 ID_073_P1100157.JPG
482 ID_073_P1100157.JPG Cropped image shape: (1863, 1315, 

529 ID_078_P1100212.JPG Cropped image shape: (1947, 1314, 3) saved quarter_reslution_tactile_img_cropped/ID_078_P1100212_cropped.JPG
530 ID_079_P1100213.JPG
530 ID_079_P1100213.JPG Cropped image shape: (1889, 1313, 3) saved quarter_reslution_tactile_img_cropped/ID_079_P1100213_cropped.JPG
531 ID_079_P1100214.JPG
531 ID_079_P1100214.JPG Cropped image shape: (1946, 1313, 3) saved quarter_reslution_tactile_img_cropped/ID_079_P1100214_cropped.JPG
532 ID_079_P1100215.JPG
532 ID_079_P1100215.JPG Cropped image shape: (1902, 1313, 3) saved quarter_reslution_tactile_img_cropped/ID_079_P1100215_cropped.JPG
533 ID_079_P1100216.JPG
533 ID_079_P1100216.JPG Cropped image shape: (1867, 1310, 3) saved quarter_reslution_tactile_img_cropped/ID_079_P1100216_cropped.JPG
534 ID_079_P1100217.JPG
534 ID_079_P1100217.JPG Cropped image shape: (1860, 1290, 3) saved quarter_reslution_tactile_img_cropped/ID_079_P1100217_cropped.JPG
535 ID_079_P1100218.JPG
535 ID_079_P1100218.JPG Cropped image shape: (1858, 1303, 

582 ID_085_P1100267.JPG Cropped image shape: (1958, 1313, 3) saved quarter_reslution_tactile_img_cropped/ID_085_P1100267_cropped.JPG
583 ID_085_P1100268.JPG
583 ID_085_P1100268.JPG Cropped image shape: (1939, 1271, 3) saved quarter_reslution_tactile_img_cropped/ID_085_P1100268_cropped.JPG
584 ID_086_P1100280.JPG
584 ID_086_P1100280.JPG Cropped image shape: (1955, 1291, 3) saved quarter_reslution_tactile_img_cropped/ID_086_P1100280_cropped.JPG
585 ID_086_P1100281.JPG
585 ID_086_P1100281.JPG Cropped image shape: (1930, 1277, 3) saved quarter_reslution_tactile_img_cropped/ID_086_P1100281_cropped.JPG
586 ID_086_P1100282.JPG
586 ID_086_P1100282.JPG Cropped image shape: (1927, 1275, 3) saved quarter_reslution_tactile_img_cropped/ID_086_P1100282_cropped.JPG
587 ID_086_P1100283.JPG
587 ID_086_P1100283.JPG Cropped image shape: (1939, 1265, 3) saved quarter_reslution_tactile_img_cropped/ID_086_P1100283_cropped.JPG
588 ID_086_P1100284.JPG
588 ID_086_P1100284.JPG Cropped image shape: (1945, 1280, 

635 ID_091_P1100325.JPG Cropped image shape: (1907, 1253, 3) saved quarter_reslution_tactile_img_cropped/ID_091_P1100325_cropped.JPG
636 ID_091_P1100326.JPG
636 ID_091_P1100326.JPG Cropped image shape: (1924, 1255, 3) saved quarter_reslution_tactile_img_cropped/ID_091_P1100326_cropped.JPG
637 ID_091_P1100327.JPG
637 ID_091_P1100327.JPG Cropped image shape: (1922, 1255, 3) saved quarter_reslution_tactile_img_cropped/ID_091_P1100327_cropped.JPG
638 ID_092_P1100328.JPG
638 ID_092_P1100328.JPG Cropped image shape: (1571, 1138, 3) saved quarter_reslution_tactile_img_cropped/ID_092_P1100328_cropped.JPG
639 ID_092_P1100329.JPG
639 ID_092_P1100329.JPG Cropped image shape: (1565, 1116, 3) saved quarter_reslution_tactile_img_cropped/ID_092_P1100329_cropped.JPG
640 ID_092_P1100330.JPG
640 ID_092_P1100330.JPG Cropped image shape: (1581, 1150, 3) saved quarter_reslution_tactile_img_cropped/ID_092_P1100330_cropped.JPG
641 ID_092_P1100333.JPG
641 ID_092_P1100333.JPG Cropped image shape: (1577, 1147, 

688 ID_098_P1100391.JPG Cropped image shape: (1597, 1132, 3) saved quarter_reslution_tactile_img_cropped/ID_098_P1100391_cropped.JPG
689 ID_098_P1100392.JPG
689 ID_098_P1100392.JPG Cropped image shape: (1614, 1128, 3) saved quarter_reslution_tactile_img_cropped/ID_098_P1100392_cropped.JPG
690 ID_098_P1100393.JPG
690 ID_098_P1100393.JPG Cropped image shape: (1676, 1189, 3) saved quarter_reslution_tactile_img_cropped/ID_098_P1100393_cropped.JPG
691 ID_098_P1100394.JPG
691 ID_098_P1100394.JPG Cropped image shape: (1582, 1096, 3) saved quarter_reslution_tactile_img_cropped/ID_098_P1100394_cropped.JPG
692 ID_098_P1100395.JPG
692 ID_098_P1100395.JPG Cropped image shape: (1668, 1187, 3) saved quarter_reslution_tactile_img_cropped/ID_098_P1100395_cropped.JPG
693 ID_099_P1100396.JPG
693 ID_099_P1100396.JPG Cropped image shape: (1584, 1122, 3) saved quarter_reslution_tactile_img_cropped/ID_099_P1100396_cropped.JPG
694 ID_099_P1100397.JPG
694 ID_099_P1100397.JPG Cropped image shape: (1594, 1115, 

To select the truck, choose a point on it. Points are input to the model in (x,y) format and come with labels 1 (foreground point) or 0 (background point). Multiple points can be input; here we use only one. The chosen point will be shown as a star on the image.